## change finderで暴騰・暴落を発見する
変化点検知

In [ ]:
%matplotlib inline
import changefinder
import pandas as pd 
from datetime import datetime as dt
import matplotlib.pyplot as plt
import numpy as np
import os


def str2datetime(tstr):
    if tstr.__class__.__name__ == "datetime":
        return tstr
    ts = tstr.split(".")[0]
    return dt.strptime(ts, "%Y-%m-%d %H:%M:%S")

def calc_cf(name, df, cf, threshold=30):
    values = df[name].values
    scores = np.array([cf.update(value) for value in values])
    print(np.where(scores>30))
    print(scores)
    # 描画
    fig, ax1 = plt.subplots(figsize=(15,5))
    ax1.set_title(name)
    ax1.plot(df.index, scores,color="red")
    ax1.set_ylabel("anom_score")
    ax2 = ax1.twinx()  # 2つのプロットを関連付ける
    ax2.plot(df.index, df[name].values)
    ax2.set_ylabel("value")
    # しきい値越えリスト
    df["score"] = scores - threshold
    for i in range(1,len(df)):
        if df.iloc[i-1]["score"] < 0 and df.iloc[i]["score"] >= 0:
            print("{}, time:{}".format(i, df.iloc[i]["timestamp"]))
            ax2.plot(df.index[i],df.iloc[i][name],"o")
        
    plt.xlim(df.index[0],df.index[-1])
    plt.show()



## Change Finder
- r: 忘却パラメータに敏感
- threthholdの調整も難しい

In [ ]:
cf = changefinder.ChangeFinder(r=0.01, order=1, smooth=6)
df = pd.read_csv("coindata.csv").dropna()
df.index = df.timestamp.apply(str2datetime)
calc_cf(name="xrp",df=df, cf=cf, threshold=15)

In [ ]:
# ARIMA(よくわからん)
# cf = changefinder.ChangeFinderARIMA(term=5,smooth=3,order=(1,0,0))
# df = pd.read_csv("coindata.csv").dropna()
# df.index = df.timestamp.apply(str2datetime)
# calc_cf(name="xem",df=df, cf=cf, threshold=30)


## Bollinger Band
- window幅の調節が必要

In [ ]:
def calc_bb(name, df, window=24):
    # ボリンジャーバンドの2σ
    plt.figure(figsize=(15,5))
    plt.title(name)
    plt.plot(df.index, df[name])
    m_ave = df[name].rolling(window=6).mean()
    m = df[name].rolling(window=window).mean()
    sigma = df[name].rolling(window=window).std()
    df["m"] = m
    df["m+2s"] = m+2*sigma
    df["m-2s"] = m-2*sigma
    df["up"] = df[name] - df["m+2s"]
    df["down"] = df["m-2s"] - df[name]
    df["up"] = m_ave - df["m+2s"]
    df["down"] = df["m-2s"] - m_ave
    
    plt.plot(df.index,m,label="{}min".format(window*10))
    plt.plot(df.index,m+2*sigma,label="$+2\sigma$",color="k",alpha=0.8,linestyle="dotted")
    plt.plot(df.index,m-2*sigma,label="$-2\sigma$",color="k",alpha=0.8,linestyle="dotted")
    plt.plot(df.index,m_ave,label="mv ave window=60min",color="green",alpha=0.4)
#     plt.plot(df.index, df[name].rolling(window=3*window).mean(),label="{}min".format(window*30))

    for i in range(1,len(df)):
        if df.iloc[i-1]["up"] < 0 and df.iloc[i]["up"] >= 0:
            print("time:{}".format(df.iloc[i]["timestamp"]))
            plt.plot(df.index[i],df.iloc[i][name],"o",color="red",markersize=10)
        if df.iloc[i-1]["down"] < 0 and df.iloc[i]["down"] >= 0:
            print("time:{}".format(df.iloc[i]["timestamp"]))
            plt.plot(df.index[i],df.iloc[i][name],"o",color="blue",markersize=10)
    plt.legend(loc=2)
    plt.show()
    
    

In [ ]:
calc_bb("xrp", df, window=36)